In [34]:
import re
import tqdm
import pandas as pd
import matplotlib.pyplot as plt

In [35]:
sale_df = pd.read_excel('../data/附件2.xlsx')

In [36]:
sale_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878503 entries, 0 to 878502
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   销售日期        878503 non-null  datetime64[ns]
 1   扫码销售时间      878503 non-null  object        
 2   单品编码        878503 non-null  int64         
 3   销量(千克)      878503 non-null  float64       
 4   销售单价(元/千克)  878503 non-null  float64       
 5   销售类型        878503 non-null  object        
 6   是否打折销售      878503 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 46.9+ MB


In [37]:
sale_df.head()

,销售日期,扫码销售时间,单品编码,销量(千克),销售单价(元/千克),销售类型,是否打折销售
0,2020-07-01,09:15:07.924,102900005117056,0.396,7.6,销售,否
1,2020-07-01,09:17:27.295,102900005115960,0.849,3.2,销售,否
2,2020-07-01,09:17:33.905,102900005117056,0.409,7.6,销售,否
3,2020-07-01,09:19:45.450,102900005115823,0.421,10.0,销售,否
4,2020-07-01,09:20:23.686,102900005115908,0.539,8.0,销售,否


In [38]:
ty_df = pd.read_excel('../data/附件1.xlsx')
item_corpus = {row[1]['单品编码']: (row[1]['单品名称'], row[1]['分类编码']) 
               for row in ty_df.iterrows()}
ty_corpus = {row[1]['分类编码']: row[1]['分类名称'] 
             for row in ty_df.iterrows()}
rev_item_corpus = {v[0]: k for k, v in item_corpus.items()}
print(list(item_corpus.items())[:5], list(ty_corpus.items())[:5], sep='\n')

[(102900005115168, ('牛首生菜', 1011010101)), (102900005115199, ('四川红香椿', 1011010101)), (102900005115625, ('本地小毛白菜', 1011010101)), (102900005115748, ('白菜苔', 1011010101)), (102900005115762, ('苋菜', 1011010101))]
[(1011010101, '花叶类'), (1011010201, '花菜类'), (1011010402, '水生根茎类'), (1011010501, '茄类'), (1011010504, '辣椒类')]


In [39]:
ty_df.head()

,单品编码,单品名称,分类编码,分类名称
0,102900005115168,牛首生菜,1011010101,花叶类
1,102900005115199,四川红香椿,1011010101,花叶类
2,102900005115625,本地小毛白菜,1011010101,花叶类
3,102900005115748,白菜苔,1011010101,花叶类
4,102900005115762,苋菜,1011010101,花叶类


In [40]:
# 使用正则表达式提取主要的单品名称，只有当括号内是数字时才分割
def extract_main_name(name):
    if re.match(r'.*\(\d+\)$', name):
        return re.split(r'\s*\(\d+\)$', name)[0]  # 分割字符串并返回第一部分
    return name

ty_df['主要名称'] = ty_df['单品名称'].apply(extract_main_name)

# 根据主要名称进行分组，并计算每个组的出现次数
name_counts = ty_df['主要名称'].value_counts()

# 筛选出同货异源的产品，即出现次数大于1的
same_goods_sources = name_counts[name_counts > 1].index.tolist()

print(same_goods_sources, name_counts, sep='\n')

['海鲜菇(袋)', '金针菇(袋)', '鲜粽叶(袋)', '净藕', '紫茄子', '黄白菜', '虫草花(盒)', '黄心菜', '蟹味菇', '海鲜菇', '小青菜', '红灯笼椒', '白玉菇', '芜湖青椒', '杏鲍菇', '鲜木耳', '七彩椒', '灯笼椒', '紫白菜', '金针菇', '青茄子', '西峡花菇', '姬菇', '西峡香菇', '圆茄子', '野藕', '高瓜', '红椒', '青杭椒']
主要名称
海鲜菇(袋)        4
金针菇(袋)        3
鲜粽叶(袋)        3
净藕            3
紫茄子           2
             ..
蔡甸藜蒿(份)       1
红珊瑚(粗叶)       1
红橡叶           1
绿牛油           1
和丰阳光海鲜菇(包)    1
Name: count, Length: 217, dtype: int64


In [41]:
temp_ty_dict = {main_name: [main_name + '(' + str(i+1) + ')' 
                          for i in range(name_counts[main_name])] 
              for main_name in same_goods_sources}
temp_idx_dict = {main_name: [{rev_item_corpus[main_name + '(' + str(i+1) + ')']:0} 
                          for i in range(name_counts[main_name])] 
              for main_name in same_goods_sources}

In [46]:
# 创建映射字典
item_to_main_name = dict(zip(ty_df['单品编码'], ty_df['主要名称']))
# 使用映射字典更新sales_df
sale_df['主要名称'] = sale_df['单品编码'].map(item_to_main_name)
sale_df.head()

,销售日期,扫码销售时间,单品编码,销量(千克),销售单价(元/千克),销售类型,是否打折销售,主要名称
0,2020-07-01,09:15:07.924,102900005117056,0.396,7.6,销售,否,泡泡椒(精品)
1,2020-07-01,09:17:27.295,102900005115960,0.849,3.2,销售,否,大白菜
2,2020-07-01,09:17:33.905,102900005117056,0.409,7.6,销售,否,泡泡椒(精品)
3,2020-07-01,09:19:45.450,102900005115823,0.421,10.0,销售,否,上海青
4,2020-07-01,09:20:23.686,102900005115908,0.539,8.0,销售,否,菜心


In [63]:
grp_sale = sale_df.groupby(['主要名称'])
for main_name, sub_id_dict_list in temp_idx_dict.items():
    main_grp = grp_sale.get_group((main_name,))
    _main_grp = sale_df.groupby(['单品编码'])
    for sub_id_dict in sub_id_dict_list:
        idx, value = list(sub_id_dict.keys())[0], list(sub_id_dict.values())[0]
        try:
            item = _main_grp.get_group((idx,))
            for _, row in item.iterrows():
                value += row['销量(千克)']
            sub_id_dict[idx] = value
        except KeyError:
            print(f'{idx} not found')
print(temp_idx_dict)

102900011011782 not found
102900011023648 not found
{'海鲜菇(袋)': [{102900011011546: 7596.0}, {106930274620090: 4248.0}, {106971533455008: 10650.0}, {106958851400125: 12894.0}], '金针菇(袋)': [{102900011001813: 12894.0}, {106949711300167: 19050.0}, {106949711300068: 15294.0}], '鲜粽叶(袋)': [{102900011034354: 1200.0}, {106971563780002: 48.0}, {106972776821582: 768.0}], '净藕': [{102900005116899: 162896.64000001104}, {102900011023976: 373.32599999999957}, {102900011035511: 2092.8000000000134}], '紫茄子': [{102900011033982: 1782.9480000000099}, {102900005116257: 81612.00599999631}], '黄白菜': [{102900011035771: 932.9760000000281}, {102900051010455: 47927.93999999648}], '虫草花(盒)': [{102900011011782: 0}, {106973223300667: 2.0}], '黄心菜': [{102900005116790: 5822.598000000073}, {102900011033920: 3765.535999999993}], '蟹味菇': [{102900005119104: 204.0}, {106956146480203: 190.0}], '海鲜菇': [{102900005119944: 1004.3300000000013}, {102900011032633: 103.78999999999998}], '小青菜': [{102900011023464: 6535.986000000108}, {10290